## DWI Processing

In [ ]:
from nipype.interfaces.io import DataSink, SelectFiles, DataGrabber # Data i/o
from nipype.interfaces.utility import IdentityInterface, Function     # utility
from nipype.pipeline.engine import Node, Workflow, MapNode, JoinNode        # pypeline engine
from nipype.interfaces.nipy.preprocess import Trim
from nipype.interfaces.fsl.epi import ApplyTOPUP, TOPUP, Eddy
from nipype.interfaces.fsl.dti import DTIFit
from nipype.interfaces.fsl.utils import Merge
from nipype.interfaces.freesurfer import Binarize
from pandas import DataFrame, Series

#set output file type for FSL to NIFTI_GZ
from nipype.interfaces.fsl.preprocess import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

# Set study variables
studyhome = '/Users/catcamacho/Box/SEA/dti_img'
raw_dir = studyhome + '/raw'
output_dir = studyhome + '/proc'
workflow_dir = studyhome + '/workflows'

subjects_list = ['Newborn']

proc_cores = 2 # number of cores of processing for the workflows
acq_file = studyhome + '/misc/acq_params.txt'
index_file = studyhome + '/misc/index.txt'

In [ ]:
## File handling Nodes

# Identity node- select subjects
infosource = Node(IdentityInterface(fields=['subject_id']),
                     name='infosource')
infosource.iterables = ('subject_id', subjects_list)

# Datasink- where our select outputs will go
substitutions = [('_subject_id_', '')]
datasink = Node(DataSink(), name='datasink')
datasink.inputs.base_directory = output_dir
datasink.inputs.container = output_dir
datasink.inputs.substitutions = substitutions

### Unwarping and preprocessing

In [ ]:
## File handling nodes

pes_template={'PE_vol': raw_dir + '/%s/dti_pe*.nii.gz'}
pes_selectfiles = Node(DataGrabber(sort_filelist=True,
                                   template = raw_dir + '/%s/dti_pe*.nii.gz',
                                   field_template = pes_template,
                                   base_directory=raw_dir,
                                   infields=['subject_id'], 
                                   template_args={'PE_vol':[['subject_id']]}), 
                       name='pes_selectfiles')

dwis_template={'dwi': raw_dir + '/{subject_id}/dti_pe0_ms103.nii.gz',
               'bvec': raw_dir + '/{subject_id}/dti_pe0_ms103.bvec', 
               'bval': raw_dir + '/{subject_id}/dti_pe0_ms103.bval'}
dwis_selectfiles = Node(SelectFiles(dwis_template), name='dwis_selectfiles')

In [ ]:
## Unwarping and ME cleaning nodes

# include only the first volume of each PE volume
trim_PEs = MapNode(Trim(end_index=1),name='trim_PEs',iterfield=['in_file'])

# merge to 1 file for topup to calculate the fieldcoef
merge_pes = Node(Merge(dimension='t',
                       merged_file='merged_pes.nii.gz'),
                 name='merge_pes')

topup = Node(TOPUP(encoding_file=acq_file), name='topup')

make_mask = Node(Binarize(min=10), name='binarize')

eddy = Node(Eddy(in_index=index_file, in_acqp=acq_file, method='jac',num_threads=2),name='eddy_correct')

fitdti = Node(DTIFit(save_tensor=True), name='fitdti')

In [ ]:
preprocflow = Workflow(name='preprocflow')
preprocflow.connect([(infosource,pes_selectfiles, [('subject_id','subject_id')]),
                     (infosource,dwis_selectfiles, [('subject_id','subject_id')]),
                     (pes_selectfiles, trim_PEs, [('PE_vol','in_file')]), 
                     (trim_PEs, merge_pes, [('out_file','in_files')]), 
                     (merge_pes, topup, [('merged_file','in_file')]),
                     
                     (dwis_selectfiles,make_mask,[('dwi','in_file')]),
                     (make_mask, eddy, [('binary_file','in_mask')]),
                     
                     (topup, eddy, [('out_fieldcoef','in_topup_fieldcoef'),
                                    ('out_movpar','in_topup_movpar')]),
                     (dwis_selectfiles, eddy, [('bvec','in_bvec'),('bval','in_bval'),('dwi','in_file')]),
                     (eddy,fitdti,[('out_corrected','dwi'),('out_rotated_bvecs','bvecs')]),
                     (dwis_selectfiles, fitdti, [('bval','bvals')]),
                     
                     (eddy,datasink, [('out_corrected','unwarped_dwi'),
                                      ('out_rotated_bvecs','rotated_bvecs'),
                                      ('out_restricted_movement_rms','dwi_restricted_movement_rms'),
                                      ('out_cnr_maps','eddy_cnr_maps')]), 
                     (fitdti, datasink, [('FA','dwi_FA')]),
                     (fitdti, datasink, [('L1','dwi_L1')]),
                     (fitdti, datasink, [('tensor','dwi_tensor')]),
                     (fitdti, datasink, [('MD','dwi_MD')])
                    ])

preprocflow.base_dir = workflow_dir
#preprocflow.write_graph(graph2use='flat')
preprocflow.run('MultiProc', plugin_args={'n_procs': 2, 'memory_gb':10})